In [1]:
!pip install torch-summary torcheval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 4.7 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset, DatasetDict
from PIL import Image
import torch
from torchvision import transforms
from transformers import ViTModel, TrainingArguments, Trainer
from torch import nn
from torchsummary import summary
from torch.utils.data import DataLoader
from safetensors.torch import load_file as safetensors_load_file
from huggingface_hub import create_repo, HfApi
import logging
import os
import json
import shutil
import csv

2024-07-14 05:25:29.346876: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-14 05:25:29.347075: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-14 05:25:29.520538: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from datasets import load_dataset

dataset = dataset = load_dataset("tranhuy/facial-beauty-rating")

Generating train split:   0%|          | 0/5500 [00:00<?, ? examples/s]

In [4]:
# Split the dataset into train and test
train_test_split = dataset['train'].train_test_split(test_size=0.2)
dataset = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'rating'],
        num_rows: 4400
    })
    test: Dataset({
        features: ['image', 'rating'],
        num_rows: 1100
    })
})

In [5]:
# Define a transform to convert PIL images to tensors
value_column_name = 'rating'
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Get max value
train_values = dataset['train'][value_column_name]
test_values = dataset['test'][value_column_name]
max_value = max(train_values + test_values)
min_value = min(train_values + test_values)
print('Max Value:', max_value)
print('Min Value:', min_value)

Max Value: 4.75
Min Value: 1.016667


In [6]:
def preprocess(example):
        example['image'] = transform(example['image'])
        return example

# Apply the preprocessing with normalization
dataset = dataset.map(preprocess, batched=False)

Map:   0%|          | 0/4400 [00:00<?, ? examples/s]

Map:   0%|          | 0/1100 [00:00<?, ? examples/s]

In [7]:
class ViTRegressionModel(nn.Module):
    def __init__(self):
        super(ViTRegressionModel, self).__init__()
        self.vit = ViTModel.from_pretrained('google/vit-base-patch16-224')
        self.fc1 = nn.Linear(self.vit.config.hidden_size, 256)
        self.fc_dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(self.vit.config.hidden_size, 1)

    def forward(self, pixel_values, labels=None):
        outputs = self.vit(pixel_values=pixel_values)
        cls_output = outputs.last_hidden_state[:, 0, :]  # Take the [CLS] token
        x = self.fc1(cls_output)
        x = self.fc_dropout(x)
        values = self.fc2(x)
        loss = None
        if labels is not None:
            loss_fct = nn.MSELoss()
            loss = loss_fct(values.view(-1), labels.view(-1))
        return (loss, values) if loss is not None else values
model = ViTRegressionModel()
summary(model, (3,224,224))

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: Failed to run torchsummary. See above stack traces for more details. Executed layers up to: [ViTModel: 1-1, ViTEmbeddings: 2-1, ViTPatchEmbeddings: 3-1, Conv2d: 4-1, Dropout: 3-2, ViTEncoder: 2-2, ViTLayer: 4-2, ViTLayer: 4-3, ViTLayer: 4-4, ViTLayer: 4-5, ViTLayer: 4-6, ViTLayer: 4-7, ViTLayer: 4-8, ViTLayer: 4-9, ViTLayer: 4-10, ViTLayer: 4-11, ViTLayer: 4-12, ViTLayer: 4-13, LayerNorm: 2-3, ViTPooler: 2-4, Linear: 3-3, Tanh: 3-4, Linear: 1-2, Dropout: 1-3]

In [ ]:
def collate_fn(batch):
        # Ensure that each item['image'] is a tensor
        pixel_values = torch.stack([torch.tensor(item['image']) for item in batch])
        labels = torch.tensor([item[value_column_name] for item in batch], dtype=torch.float).unsqueeze(1)
        return {'pixel_values': pixel_values, 'labels': labels}

In [ ]:
num_train_epochs = 5
training_args = TrainingArguments(
    output_dir='./model',
    evaluation_strategy="epoch",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=num_train_epochs,
    learning_rate=0.001,
    save_steps=10,
    save_total_limit=2,
    logging_steps=10,
    remove_unused_columns=False,
    resume_from_checkpoint=True,
)

train_dataloader = DataLoader(dataset['train'], batch_size=32, shuffle=True, collate_fn=collate_fn)
eval_dataloader = DataLoader(dataset['test'], batch_size=32, shuffle=False, collate_fn=collate_fn)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    data_collator=collate_fn,
)

# Add logging to inspect the model outputs and labels
def compute_metrics(p):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    labels = p.label_ids
    logger.info(f"Predictions: {preds[:5]}")
    logger.info(f"Labels: {labels[:5]}")
    mse = ((preds - labels) ** 2).mean().item()
    return {"mse": mse}

trainer.compute_metrics = compute_metrics

trainer.train()
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")